In [1]:
file = open('demofile.dots', 'r', encoding='UTF-8')

In [2]:
lines = file.readlines()

In [3]:
import re
declarator = r":[_A-Za-z][A-Za-z0-9]*:"
literals = r"\"[A-Za-z0-9\;\,\.\-\:\^\*\ ]*\""
variables = r"[A-Za-z][A-Za-z0-9_]*"
functional_symbols = [
    {
        "name":"open_parentheses",
        "matcher":r"\("
    },
    {
        "name":"close_parentheses",
        "matcher":r"\)"
    },
    {
        "name":"open_curly",
        "matcher":r"\{"
    },
    {
        "name":"close_curly",
        "matcher":r"\}"
    },
    {
        "name":"equals_assign",
        "matcher":r" :=: "
    },
        {
        "name": "string",
        "converts_to":str,
        "matcher":r"string "
    },
    {
        "name": "int",
        "converts_to": int,
        "matcher": r"int "
    },
    {
        "name": "float",
        "converts_to": float,
        "matcher": r"float "
    }
]
variable_types = [
    {
        "name": "string",
        "converts_to":str,
        "matcher":r"string "
    },
    {
        "name": "int",
        "converts_to": int,
        "matcher": r"int "
    },
    {
        "name": "float",
        "converts_to": float,
        "matcher": r"float "
    }
]
comment = r"\..*\."


In [4]:
from operator import indexOf


def filler_inLine(found_group, line, lines):
    filler = ""
    indexToReplace = indexOf(lines,line)
    for i in range(0,len(found_group)):
        filler+=" "
    line = line.replace(found_group,filler)
    lines[indexToReplace] = line
    return line, lines

In [5]:
from operator import indexOf
import re
stack = []
for index in range(0,len(lines)):
    #* Define a linha a ser analisada
    line = lines[index]
    #* Encontra comentários de uma linha
    if found_comment:
       line, lines = filler_inLine(found_comment.group(),line,lines)
       print(line)
    #* Encontra um literal
    found_literal = re.search(literals,line)
    if found_literal:
        stack.append(
            {
                'line': indexOf(lines,line),
                'position': found_literal.span(),
                'match': found_literal.group()
            }
        )
        line, lines = filler_inLine(found_literal.group(), line, lines)
    
    #* Encontra um declarador
    found_declarator = re.search(declarator,line)
    if found_declarator:
        stack.append(
            {
                'line': indexOf(lines,line),
                'position': found_declarator.span(),
                'match': found_declarator.group()
            }
        )
        line, lines = filler_inLine(found_declarator.group(),line,lines)
    
    #* Simbolos Funcionais
    for functional_symbol in functional_symbols:
        found_functional_symbol = re.search(functional_symbol["matcher"], line)
        if found_functional_symbol:
            stack.append(            {
                'line': indexOf(lines,line),
                'position': found_functional_symbol.span(),
                'match': found_functional_symbol.group()
            })
            line, lines = filler_inLine(found_functional_symbol.group(),line,lines)
            
    #* Encontra uma variavel
    found_variable = re.search(variables,line)
    if found_variable:
        stack.append(            {
            'line': indexOf(lines,line),
            'position': found_variable.span(),
            'match': found_variable.group()
        })
        line, lines = filler_inLine(found_variable.group(),line,lines)
    found_comment = re.search(comment, line)
            
print(stack)

NameError: name 'found_comment' is not defined

In [ ]:
file = open('demofile.dots', 'r', encoding='UTF-8')
oc_lines = file.readlines()

In [ ]:
stack.sort(key=lambda x: x['position'][0])
stack.sort(key=lambda x: x['line'])

print(stack)

#for item in stack:
    #print(str(oc_lines[item['line']])[item['position'][0]:item['position'][1]])

[{'line': 0, 'position': (0, 3), 'match': ':m:'}, {'line': 0, 'position': (3, 4), 'match': '('}, {'line': 0, 'position': (4, 5), 'match': ')'}, {'line': 0, 'position': (5, 6), 'match': '{'}, {'line': 1, 'position': (4, 11), 'match': 'string '}, {'line': 1, 'position': (11, 16), 'match': 'hello'}, {'line': 1, 'position': (16, 21), 'match': ' :=: '}, {'line': 1, 'position': (21, 38), 'match': '"Hello :m: World"'}, {'line': 2, 'position': (4, 7), 'match': ':p:'}, {'line': 2, 'position': (7, 8), 'match': '('}, {'line': 2, 'position': (8, 13), 'match': 'hello'}, {'line': 2, 'position': (13, 14), 'match': ')'}, {'line': 3, 'position': (6, 10), 'match': 'aqui'}, {'line': 4, 'position': (0, 1), 'match': '}'}]


In [8]:
# @ é vazio
def genFirst(input:str, non_terminals, terminals, productions):
    first = set()
    if input in non_terminals:
        for alternative in productions[input]:
            first = first | genFirst(alternative, non_terminals, terminals, productions)
    elif input in terminals:
        first = {input}
    elif input=='' or input=='@':
        first = {'@'}
    else:
        first_2 = genFirst(input[0], non_terminals, terminals, productions)
        if '@' in first_2:
            i = 1
            while '@' in first_2:
                first = first | (first_2 - {'@'})
                if input[i:] in terminals:
                    first = first | {input[i:]}
                    break
                elif input[i:] == '':
                    first = first | {'@'}
                    break
                first_2 = first(input[i:])
                first = first | first_2 - {'@'}
                i += 1
        else:
            first = first | first_2
    return first

In [ ]:
def getFollow(nT, starting_symbol, productions):
    follow = set()
    prods = productions.items()
    if nT==starting_symbol:
        follow = follow | {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for char in alt:
                if char==nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str=='':
                        if nt==nT:
                            continue
                        else:
                            follow = follow | getFollow(nt)
                    else:
                        follow_2 = genFirst(following_str)
                        if '@' in follow_2:
                            follow = follow | follow_2-{'@'}
                            follow = follow | getFollow(nt)
                        else:
                            follow = follow | follow_2
    return follow

In [10]:
terminals = {'a','b','c'}
non_terminals = {'S','A','B','C'}
productions = {'S':['A','B','C','@'],'A':['aA','@'],'B':['bB','@'],'C':['cC','@']}

print(genFirst('S',non_terminals,terminals,productions))
print(genFirst('A',non_terminals,terminals,productions))
print(genFirst('B',non_terminals,terminals,productions))
print(genFirst('C',non_terminals,terminals,productions))

{'@', 'c', 'b', 'a'}
{'@', 'a'}
{'@', 'b'}
{'@', 'c'}
